# Limpieza de datos

In [142]:
#Importamos librerías a utilizar
import pandas as pd #Para trabajo con datos y dataframes
import requests #para solicitud de web scraping
from bs4 import BeautifulSoup #para conversión a html y extracción de información concreta de la página.
import glob #Para importe masivo de archivos csv.
from unidecode import unidecode #para eliminación de tildes en observaciones.
import numpy as np
from datetime import datetime
import re

fecha_scraping = datetime.now().strftime("%Y-%m-%d").replace("-", "_")

In [143]:
#Importamos la data
df = pd.read_csv(f"../data/raw/ofertas_{fecha_scraping}_raw.csv", encoding="utf-8")

In [144]:
#Limpiamos los valores de precio original y precio de oferta para que queden en formato numérico de número flotante.
df["precio_original"] = df["precio_original"].str.replace("$", "", regex=False).str.replace(".", "", regex=False).str.replace(",", ".", regex=False).astype(float)
df["precio_oferta"] = df["precio_oferta"].str.replace("$", "", regex=False).str.replace(".", "", regex=False).str.replace(",", ".", regex=False).astype(float)
#Creamos una variable asociada al porcentaje de descuento que fue aplicado.
porcentaje_descuento = (1 - (df["precio_oferta"] / df["precio_original"])).round(2)
idx = df.columns.get_loc("precio_oferta")
df.insert(idx + 1, "porcentaje_descuento", porcentaje_descuento)
#Eliminamos registros con porcentaje de descuento negativo, ya que en realidad estos no están con descuento activo al visitar el link.
df = df[df["porcentaje_descuento"] > 0]

#Limpiamos los valores de la variable "peso" para conservarlo como número flotante en kg.
df["peso"] = df["peso"].str.replace(" kg", "", regex=False).str.replace(".", "", regex=False).str.replace(",", ".", regex=False).astype(float)
df = df.rename(columns={"peso": "peso_kg"})

#Separamos la variable dimensiones en largo, ancho y grosor del libro.
# función auxiliar para limpiar y corregir
def limpiar_valor(i):
    v = float(i.replace(",", ".").replace("cm", "").strip())
    if 100 <= v <= 999:  # si tiene tres cifras
        v *= 0.1
    return v

# aplicar la transformación
nuevas = df["dimensiones"].apply(
    lambda x: pd.Series(
        sorted([limpiar_valor(i) for i in x.split("×")], reverse=True)
    )
)

# insertar las nuevas columnas después de "dimensiones"
idx = df.columns.get_loc("dimensiones")
df.insert(idx + 1, "largo", nuevas[0])
df.insert(idx + 2, "ancho", nuevas[1])
df.insert(idx + 3, "grosor", nuevas[2])

#Estandarizamos los textos de las variables de tipo string. Eliminamos tildes y transformamos los textos a minúsculas.
df["titulo"] = df["titulo"].astype(str).apply(lambda x: unidecode(x).lower())
df["categoria"] = df["categoria"].astype(str).apply(lambda x: unidecode(x).lower())
df["macrocategoria"] = df["macrocategoria"].astype(str).apply(lambda x: unidecode(x).lower())
df["autor"] = df["autor"].astype(str).apply(lambda x: unidecode(x).lower())
df["editorial"] = df["editorial"].astype(str).apply(lambda x: unidecode(x).lower())
df["encuadernacion"] = df["encuadernacion"].astype(str).apply(lambda x: unidecode(x).lower())

#descripcion_limpia = re.sub(r'(?<![A-Z])(?=[A-Z])', ' ', descripcion).strip()
#descripcion_limpia = re.sub(r'\s+', ' ', descripcion_limpia)

def limpiar_descripcion(texto):
    if pd.isna(texto):  # Maneja valores NaN
        return texto
    
    # 1️⃣ Insertar espacio antes de mayúscula que sigue a minúscula o puntuación
    texto_limpia = re.sub(r'(?<![A-Z])(?=[A-Z])', ' ', texto).strip()
    
    # 2️⃣ Quitar espacios después de signos iniciales ¿ y ¡
    texto_limpia = re.sub(r'([¿¡])\s+', r'\1', texto_limpia)
    
    # 3️⃣ Normalizar espacios
    texto_limpia = re.sub(r'\s+', ' ', texto_limpia)
    
    return texto_limpia

df['descripcion'] = df['descripcion'].apply(limpiar_descripcion)


#creamos la variable volumen para reemplazar a las de dimensiones, ya que pueden ser muy correlacionadas.
df["volumen"] = df["largo"] * df["ancho"] * df["grosor"] 


In [145]:
print(df["idioma"].value_counts())
#dado que hay tan poca variabilidad en cuanto a la variable idioma, se opta por eliminar la variable, ya que no es de utilidad.
#Por otra parte, dado que ya separamos las dimensiones en tres variables distintas, procedemos a eliminar dicha variable, ya que es redundante.
df = df.drop(columns=["idioma","dimensiones", "info_adicional"])

idioma
Español                                3078
Inglés                                   74
Bilingüe Español – Ingles                 3
Plurilingüe                               2
Trilingüe Español / Inglés / Alemán       2
Bilingüe Español Ingles                   1
Bilingüe Español – Chino                  1
Name: count, dtype: int64


In [146]:
#Eliminamos registros cuyos valores para las variables ano_edicion, paginas sean NaN (None).
#Esto debido a que dichas variables se consideran de gran relevancia para un análisis.
df = df.dropna(subset=['ano_edicion', 'paginas']).reset_index(drop=True) #dropeamos alrededor de 80 observaciones.

#Se opta por no eliminar ninguna columna en este jupyter notebook, sino que será en PowerBI.

#Dependiendo del foco que se tenga, las variables asociadas al título, autor, editorial, descripcion, link pueden adoptar mayor o menor importancia para un análisis.

In [147]:
df.dtypes #observamos la clase correspondiente a cada variable. Está todo correcto.

titulo                   object
categoria                object
macrocategoria           object
disponibles               int64
precio_original         float64
precio_oferta           float64
porcentaje_descuento    float64
autor                    object
editorial                object
encuadernacion           object
peso_kg                 float64
largo                   float64
ancho                   float64
grosor                  float64
ano_edicion             float64
paginas                 float64
descripcion              object
link                     object
fecha_extraccion         object
volumen                 float64
dtype: object

In [148]:
#Verificamos la presencia de valores NaN dentro del dataframe
print(df.isnull().any())
print(df["grosor"].isna().value_counts())
#Sólo la variable "grosor" presenta valores NaN.
#Debido a que aún dicha variable no se considera de gran relevancia y además sólo presenta 3 registros faltantes, se opta por conservarlos.

#Se opta por eliminarlos, más fácil
df = df.dropna(subset=['grosor'])
print(df["grosor"].isna().value_counts())

titulo                  False
categoria               False
macrocategoria          False
disponibles             False
precio_original         False
precio_oferta           False
porcentaje_descuento    False
autor                   False
editorial               False
encuadernacion          False
peso_kg                 False
largo                   False
ancho                   False
grosor                   True
ano_edicion             False
paginas                 False
descripcion             False
link                    False
fecha_extraccion        False
volumen                  True
dtype: bool
grosor
False    3136
True        4
Name: count, dtype: int64
grosor
False    3136
Name: count, dtype: int64


In [149]:
df #observamos un poco del dataframe
#df.loc[3647,"link"]

,titulo,categoria,macrocategoria,disponibles,precio_original,precio_oferta,porcentaje_descuento,autor,editorial,encuadernacion,peso_kg,largo,ancho,grosor,ano_edicion,paginas,descripcion,link,fecha_extraccion,volumen
0,"general schneider : un hombre de honor, un cri...",historia politica de chile,politica,10,19000.0,18050.0,0.05,"schneider arce, victor",entre zorros y erizos,rustico,0.275,23.0,15.0,1.0,2025.0,172.0,"GENERAL SCHNEIDER : UN HOMBRE DE HONOR, UN CRI...",https://feriachilenadellibro.cl/producto/97895...,2025_08_30,345.00
1,sapiens: de animales a dioses,historia,historia,10,16000.0,12000.0,0.25,"harari, yuval noah",debolsillo,rustico/bolsillo,0.330,19.0,12.5,3.0,2024.0,493.0,SAPIENS: DE ANIMALES A DIOSES De la mano de un...,https://feriachilenadellibro.cl/producto/97862...,2025_08_30,712.50
2,la magia de ser sofia (sofia 1),novela romantica,literatura,10,22000.0,16500.0,0.25,"benavent, elisabet",suma de letras,rustico,0.605,23.0,15.0,2.5,2025.0,528.0,LA MAGIA DE SER SOFIA ( SOFIA 1) Sofía tiene t...,https://feriachilenadellibro.cl/producto/97898...,2025_08_30,862.50
3,cuentos completos 2 - cortazar,contemporanea,literatura,10,19000.0,14250.0,0.25,"cortazar, julio",debolsillo,rustico/bolsillo,0.585,19.0,12.5,4.0,2025.0,799.0,CUENTOS COMPLETOS 2 – CORTAZAR En este volumen...,https://feriachilenadellibro.cl/producto/97884...,2025_08_30,950.00
4,ficciones,contemporanea,literatura,10,12000.0,9000.0,0.25,"borges, jorge luis",debolsillo,rustico/bolsillo,0.173,19.0,12.5,1.5,2011.0,224.0,"FICCIONES Pensé en un laberinto, en un sinuoso...",https://feriachilenadellibro.cl/producto/97895...,2025_08_30,356.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3135,tragar veneno,futbol,deportes,4,17900.0,12900.0,0.28,"catalan, esteban",tusquets editores,rustico,0.263,22.5,15.0,1.0,2023.0,178.0,TRAGAR VENENO Poco tiene que ver la alegría co...,https://feriachilenadellibro.cl/producto/97895...,2025_08_30,337.50
3136,que te paso?,autoayuda,autoayuda,5,20900.0,14900.0,0.29,"winfrey, oprah / perry, bruce",zenith,rustico,0.551,23.0,15.0,1.5,2023.0,384.0,QUE TE PASO? ¿Alguna vez te has preguntado: «¿...,https://feriachilenadellibro.cl/producto/97895...,2025_08_30,517.50
3137,loki : donde la malicia yace,libros juveniles,libros juveniles,3,17900.0,12900.0,0.28,"lee, mackenzi",planeta editorial,rustico,0.532,21.0,13.0,2.5,2023.0,472.0,LOKI : DONDE LA MALICIA YACE Esta historia se ...,https://feriachilenadellibro.cl/producto/97895...,2025_08_30,682.50
3138,el fin de la inflacion,economia y negocios,economia y negocios,4,16500.0,11900.0,0.28,"milei, javier",planeta editorial,rustico,0.240,21.0,13.5,1.2,2023.0,184.0,EL FIN DE LA INFLACION Plantándole cara a la c...,https://feriachilenadellibro.cl/producto/97895...,2025_08_30,340.20


In [150]:
"""
#reasignamos algunas macrocategorías que presentan muy poca frecuencia
df.loc[df["macrocategoria"].str.contains(r"\blenguas extranjeras y nativas\b", case=False, na=False, regex=True), "macrocategoria"] = "ciencias de la comunicacion"

#dudoso juntar ingenieria y tecnologia
df.loc[df["macrocategoria"].str.contains(r"\binformatica e internet\b", case=False, na=False, regex=True), "macrocategoria"] = "tecnologia"
df.loc[df["macrocategoria"].str.contains(r"\btecnologia\b", case=False, na=False, regex=True), "macrocategoria"] = "ingenieria y tecnologia"
df.loc[df["macrocategoria"].str.contains(r"\bingenieria\b", case=False, na=False, regex=True), "macrocategoria"] = "ingenieria y tecnologia"

df.loc[df["macrocategoria"].str.contains(r"\bliteratura escolar\b", case=False, na=False, regex=True), "macrocategoria"] = "libros infantiles"
df.loc[df["macrocategoria"].str.contains(r"\btextos de estudio\b", case=False, na=False, regex=True), "macrocategoria"] = "libros juveniles"
df.loc[df["macrocategoria"].str.contains(r"\bentretencion\b", case=False, na=False, regex=True), "macrocategoria"] = "ocio y hobbies"

df.loc[df["macrocategoria"].str.contains(r"\barquitectura y urbanismo\b", case=False, na=False, regex=True), "macrocategoria"] = "musica-cine-ballet-teatro-arquitectura-urbanismo-geografia"
df.loc[df["macrocategoria"].str.contains(r"\bmusica-cine-ballet-teatro\b", case=False, na=False, regex=True), "macrocategoria"] = "musica-cine-ballet-teatro-arquitectura-urbanismo-geografia"
df.loc[df["macrocategoria"].str.contains(r"\bgeografia\b", case=False, na=False, regex=True), "macrocategoria"] = "musica-cine-ballet-teatro-arquitectura-urbanismo-geografia"

df.loc[df["macrocategoria"].str.contains(r"\bsexologia\b", case=False, na=False, regex=True), "macrocategoria"] = "psicologia y pedagogia"
df.loc[df["macrocategoria"].str.contains(r"\besoterismo\b", case=False, na=False, regex=True), "macrocategoria"] = "medicina alternativa"
df.loc[df["macrocategoria"].str.contains(r"\bpsicologia y pedagogia\b", case=False, na=False, regex=True), "macrocategoria"] = "psicologia, pedagogia y medicina alternativa"
"""

'\n#reasignamos algunas macrocategorías que presentan muy poca frecuencia\ndf.loc[df["macrocategoria"].str.contains(r"\x08lenguas extranjeras y nativas\x08", case=False, na=False, regex=True), "macrocategoria"] = "ciencias de la comunicacion"\n\n#dudoso juntar ingenieria y tecnologia\ndf.loc[df["macrocategoria"].str.contains(r"\x08informatica e internet\x08", case=False, na=False, regex=True), "macrocategoria"] = "tecnologia"\ndf.loc[df["macrocategoria"].str.contains(r"\x08tecnologia\x08", case=False, na=False, regex=True), "macrocategoria"] = "ingenieria y tecnologia"\ndf.loc[df["macrocategoria"].str.contains(r"\x08ingenieria\x08", case=False, na=False, regex=True), "macrocategoria"] = "ingenieria y tecnologia"\n\ndf.loc[df["macrocategoria"].str.contains(r"\x08literatura escolar\x08", case=False, na=False, regex=True), "macrocategoria"] = "libros infantiles"\ndf.loc[df["macrocategoria"].str.contains(r"\x08textos de estudio\x08", case=False, na=False, regex=True), "macrocategoria"] = "

In [151]:
#Reasignamos la macrocategoría de literatura debido a su amplia variedad de títulos.
frecuencias = df[df["macrocategoria"]=="literatura"]["categoria"].value_counts()
#Utilizaremos la respectiva categoría a los títulos cuya categoría tenga una frecuencai mayor o igual a 10. 
#Por otra parte reasignaremos como "otra literatura" a los que tengan una menor frecuencia
categorias_frecuentes = frecuencias[frecuencias >= 10].index
df.loc[(df["macrocategoria"] == "literatura") & (df["categoria"].isin(categorias_frecuentes)),"macrocategoria"] = df["categoria"]
df.loc[(df["macrocategoria"] == "literatura") & (~df["categoria"].isin(categorias_frecuentes)), "macrocategoria"] = "otra literatura"

#Con el siguiente código, identificaremos los títulos de literaturas cuyas categorías fueron modificadas y se le agregará el prefijo "literatura ".
#En caso de ya presentarlo, no se le agrega para no redundar. Se le agrega srt.startswith para identificar los títulos que ya presentan el prefijo.
mask = df["macrocategoria"].isin(categorias_frecuentes)
df.loc[mask, "macrocategoria"] = "literatura " + df.loc[mask, "macrocategoria"].where(
    df.loc[mask, "macrocategoria"].str.lower().str.startswith("literatura"),
    ""
) + df.loc[mask, "macrocategoria"]

In [152]:
df["macrocategoria"].value_counts()

macrocategoria
literatura contemporanea           334
autoayuda                          220
libros juveniles                   206
literatura clasicos universales    149
literatura autores chilenos        141
                                  ... 
ocio y hobbies                       3
geografia                            2
informatica e internet               2
ciencias de la comunicacion          1
sexologia                            1
Name: count, Length: 63, dtype: int64

In [153]:
df[df["macrocategoria"].str.contains("literatura", case=False, na=False, regex=True)]["categoria"].value_counts().head(30)


categoria
contemporanea                       334
clasicos universales                149
autores chilenos                    141
narrativa extranjera                111
novela historica                    103
novela romantica                     85
ciencia ficcion                      74
comic y novelas graficas             65
novela negra                         62
narrativa latinoamericana            56
genero policial                      55
misterio y terror                    55
novela                               48
novela contemporanea                 43
thrillers                            39
ficcion                              35
narrativa fantastica                 31
poesia                               23
ensayo                               23
narrativa historica                  22
literatura fantastica                18
narrativa espanola                   16
memorias- diarios- cronicas- etc     15
cuento                               13
narrativa erotica             

In [154]:
df[df["titulo"]=="en unidad con la vida : pensamientos inspiradores para todos los dias"]
df[df["titulo"]=="doppelganger : un viaje al mundo del espejo"]

,titulo,categoria,macrocategoria,disponibles,precio_original,precio_oferta,porcentaje_descuento,autor,editorial,encuadernacion,peso_kg,largo,ancho,grosor,ano_edicion,paginas,descripcion,link,fecha_extraccion,volumen
2773,doppelganger : un viaje al mundo del espejo,sociologia,ciencias sociales,9,28900.0,20900.0,0.28,"klein, naomi",paidos,rustico,0.738,24.0,22.5,16.0,2024.0,496.0,DOPPELGANGER : UN VIAJE AL MUNDO DEL ESPEJO Na...,https://feriachilenadellibro.cl/producto/97895...,2025_08_30,8640.0


In [155]:
df[df["grosor"]==15]

,titulo,categoria,macrocategoria,disponibles,precio_original,precio_oferta,porcentaje_descuento,autor,editorial,encuadernacion,peso_kg,largo,ancho,grosor,ano_edicion,paginas,descripcion,link,fecha_extraccion,volumen
301,la historia de zoe n 04/06 (ne),ciencia ficcion,literatura ciencia ficcion,3,20900.0,14900.0,0.29,"scalzi, john",minotauro,rustico,0.362,23.0,22.5,15.0,2023.0,320.0,LA HISTORIA DE ZOE N 04/06 ( NE) “ Sigue la hi...,https://feriachilenadellibro.cl/producto/97884...,2025_08_30,7762.5


In [156]:
df["porcentaje_descuento"].sort_values()

1412    0.04
765     0.04
0       0.05
1325    0.05
76      0.05
        ... 
2162    0.74
2153    0.74
169     0.75
2422    0.76
3130    0.79
Name: porcentaje_descuento, Length: 3136, dtype: float64

In [157]:
df["largo"].sort_values()

2356    13.5
864     15.0
1034    15.3
1403    15.5
2779    16.0
        ... 
1968    30.0
1974    30.0
1086    30.0
1328    31.0
970     33.0
Name: largo, Length: 3136, dtype: float64

In [158]:
df[df["categoria"].str.contains("comunicacion", case=False, na=False, regex=True)]

,titulo,categoria,macrocategoria,disponibles,precio_original,precio_oferta,porcentaje_descuento,autor,editorial,encuadernacion,peso_kg,largo,ancho,grosor,ano_edicion,paginas,descripcion,link,fecha_extraccion,volumen
817,variacion y significado y discurso,ciencias de la comunicacion,ciencias de la comunicacion,2,26900.0,12400.0,0.54,"lavandera, beatriz",paidos,rustico,0.52,22.0,13.0,1.0,2014.0,372.0,VARIACION Y SIGNIFICADO Y DISCURSO La primera ...,https://feriachilenadellibro.cl/producto/97895...,2025_08_30,286.0
1582,seguridad en las instalaciones de telecomunica...,telecomunicaciones,ingenieria,1,7000.0,5400.0,0.23,"cocera, julian",paraninfo thomson,rustico,0.70,29.5,21.0,1.4,2004.0,248.0,SEGURIDAD EN LAS INSTALACIONES DE TELECOMUNICA...,https://feriachilenadellibro.cl/producto/97884...,2025_08_30,867.3
3060,television digital : mpeg-1 mpeg-2,telecomunicaciones,tecnologia,2,7000.0,4500.0,0.36,"benoit, herve",paraninfo,rustico,0.35,24.0,17.0,1.0,2004.0,179.0,TELEVISION DIGITAL : MPEG-1 MPEG-2 Describe lo...,https://feriachilenadellibro.cl/producto/97884...,2025_08_30,408.0


In [159]:
#Exportamos el dataframe 
df.to_csv(f"../data/processed/ofertas_{fecha_scraping}.csv", index=False, sep=';', decimal=',', encoding="utf-8")

In [160]:
texto = df.loc[78, "descripcion"]
texto

'FUEGO Y SANGRE Su emblema es un dragón de tres cabezas, y su lema, Fuego y Sangre. El nuevo libro de George R. R. Martin narra la fascinante historia de los Targaryan, la dinastía que reinó en Poniente 300 años antes de lo narrado en « Canción de hielo y fuego», la saga que inspiró la serie de HBO Juego de Tronos. Siglos antes de que tuvieran lugar los acontecimientos que se relatan en « Canción de hielo y fuego», la casa Targaryen, la única dinastía de señores dragón que sobrevivió a la Maldición de Valyria, se asentó en la isla de Rocadragón. Aquí tenemos el primero de los dos volúmenes en el que el autor de Juego de tronos nos cuenta, con todo lujo de detalles, la historia de tan fascinante familia: empezando por Aegon I Targaryen, creador del icónico Trono de Hierro, y seguido por el resto de las generaciones de Targaryens que lucharon con fiereza por conservar el poder, y el trono, hasta la llegada de la guerra civil que casi acaba con ellos. ¿Qué pasó realmente durante la Danza 